In [61]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [75]:
def scrape_players_value():
    playersList, valuesList, clubsList, nationsList = [],[],[],[]

    headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    for k in range(2008,2016):
        for i in range(0,10):
            pages = f'https://www.transfermarkt.co.uk/transfers/transferrekorde/statistik/top/saison_id/{k}/plus/1/galerie/0/page/{i+1}'
            pageTree = requests.get(pages, headers=headers)
            pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

            players = pageSoup.find_all("a", {"class": "spielprofil_tooltip"})
            values = pageSoup.find_all("td", {"class": "rechts hauptlink"})
            clubs = pageSoup.find_all('table', {'class': 'inline-table'})
            club_names, nation_names = [], []
            for i, club in enumerate(clubs):
                if (i+1)%3 == 0:
                    club_names.append(club.find_all('img', {'class':'tiny_wappen'})[0]['alt'])
                    nation_names.append(club.find_all('img', {'class':'flaggenrahmen'})[0]['alt'])

            for j in range(0,25):
                playersList.append(players[j].text)
                valuesList.append(values[j].text)
                clubsList.append(club_names[j])
                nationsList.append(nation_names[j])
                
    df = pd.DataFrame({"Players":playersList,"Values":valuesList, 'Clubs':clubsList, 'Nations':nationsList})

    start = np.array([[f'{year}']*250 for year in range(2008,2016)]).flatten()
    end = np.array([[f'{year}']*250 for year in range(2009,2017)]).flatten()
    df['Start'] = start
    df['End'] = end
    
    import re
    def money(x):
        if re.search(pattern, x) is not None:
            return float(re.search(pattern, x)[0])
        else:
            return 0

    df['Values'] = df['Values'].apply(lambda x: money(x))
    
    return df

In [76]:
df = scrape_players_value()
df.to_csv("./values.csv")

In [77]:
df

,Players,Values,Clubs,Nations,Start,End
0,Robinho,38.70,Manchester City,England,2008,2009
1,Dimitar Berbatov,34.20,Manchester United,England,2008,2009
2,Dani Alves,31.95,FC Barcelona,Spain,2008,2009
3,Danny,27.00,Zenit St. Petersburg,Russia,2008,2009
4,Klaas-Jan Huntelaar,24.30,Real Madrid,Spain,2008,2009
5,Ricardo Quaresma,22.14,Inter Milan,Italy,2008,2009
6,Ronaldinho,21.74,AC Milan,Italy,2008,2009
7,Robbie Keane,21.60,Liverpool FC,England,2008,2009
8,Jô,21.60,Manchester City,England,2008,2009
9,Amauri,20.52,Juventus FC,Italy,2008,2009


In [78]:
df_match = pd.read_csv('./matches.csv')
df_match.head()

,Unnamed: 0,X,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
